In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import listdir
import pdb
import scipy.signal
import scipy.integrate
import scipy.interpolate
import h5py

In [2]:
#save_dir = r'/home/william/Dropbox/Materials Internship/Presentations/PMNPT Indirect CSVs/'

In [3]:
h5_file = r'/home/william/Dropbox/Materials Internship/Data/indirect_data.h5'
f = h5py.File(h5_file,'r')


In [4]:
grp = f['Crystals/PZT/PLZTcooling']
#grp = f['Crystals/PZT/PLZT_new_doping']

#grp = f['Crystals/PZT/No_doping']
#grp = f['Crystals/PMNPT/111/heating2']
grp_name = '_'.join((grp.name).split('/')[-2:])

In [5]:
def last_first_diff(arr):
    return arr[-1] - arr[0]

In [6]:
def shift_slice(the_slice,n):
    try:
        the_slice = the_slice[0]
    except:
        pass
    return slice(the_slice.start + n, the_slice.stop + n)

In [7]:
def info_loss(arr,n):
    for n in np.arange(n):
        arr = np.correlate(arr,np.ones(2)/2)
    return arr

In [8]:
def findZeros(V):
    shift = np.mean((np.max(V),np.min(V)))
    V = V - shift
    V = np.abs(V)

    zero_threshold = np.max(V)/10
    indices = np.where(V<zero_threshold)[0]
    
    zero_indices = np.array([])
    minimum_spacing = (np.max(indices) - np.min(indices)) / 10
    for a in indices:        
        cluster = indices[np.abs(indices-a)<minimum_spacing]
        new_ind = cluster[np.argmin(V[cluster])]
        zero_indices = np.append(zero_indices,new_ind)
        
    zero_indices = np.unique(zero_indices)
    

    
    try:
        assert zero_indices.size == 7
    except:
        if zero_indices.size == 6:
            zero_indices = np.append(zero_indices,V.size-1)
        else:
            raise ZeroDivisionError
    
    return zero_indices.astype('int32')
    

    

In [9]:
def voltage_slice(V,start,stop):
    
    start_index = voltage_index(V,start)
    stop_index = voltage_index(V,stop)
    
    try:
        assert stop_index>start_index+5
    except:
        pdb.set_trace()
    
    return np.index_exp[start_index:(stop_index+1)]

In [10]:
def voltage_index(V,ind):
    zero_indices = findZeros(V)
    V = np.abs(V)
    
    
    if ind % 2 == 0:
        index = zero_indices[ind//2]

    if ind % 2 == 1:
        index1 = zero_indices[(ind-1)//2]
        index2 = zero_indices[(ind+1)//2]
        index = index1 + np.argmax(V[index1:index2])

    return index

In [11]:
def leakage_correction(P,V,t,diagnose=True,no_correction=False,major_slice=False):
    #P,V,t are arrays in the same order
    #Units as given. No corrections

    
    #Finding R as a function of t and V
    minor_branch_plus = voltage_slice(V,5,7)
    R_plus = find_R2(P[minor_branch_plus],V[minor_branch_plus],t[minor_branch_plus])
    
   
    minor_branch_minus = voltage_slice(V,9,12)
    R_minus = find_R2(P[minor_branch_minus],V[minor_branch_minus],t[minor_branch_minus])    
    
    R = (R_plus+R_minus)/2
    R = 0.8*R
    
    
    #This method works fine for the ferroelectric phase, not so much in the paraelectric phase.
    #This method might work better:
    minor_branch_plus = voltage_slice(V,5,8)
    R_plus_array = find_R2(P[minor_branch_plus],V[minor_branch_plus],t[minor_branch_plus])
    R_plus = np.median(R_plus_array)
    
    minor_branch_minus = voltage_slice(V,9,12)
    R_minus_array = find_R2(P[minor_branch_minus],V[minor_branch_minus],t[minor_branch_minus])
    R_minus = np.median(R_minus_array)
    
    R = (R_plus+R_minus)/2
    
    #This method ^ doesn't work very well for the paraelectric phase either
    #Now let's try to set the curvature equal to zero by assuming that the leakage 
    #is the only contribution to curvature (this is the case at least in the paraelectric phase)
    
    #upper
    upper_slice = voltage_slice(V,5,6) #Normally 1,2
    Pu = np.copy(P[upper_slice])
    Vu = np.copy(V[upper_slice])
    tu = np.copy(t[upper_slice])
    
    VInt = scipy.integrate.cumtrapz(Vu,tu)
    VInt = np.hstack(([0],VInt))
    
    
    curvInt = getCurve(Vu,VInt)
    curvP = getCurve(Vu,Pu)

    correction = 1.2
    R = correction * curvInt / curvP
    
    

    
    #Alternative leakage correction based on the legs touching
    n = voltage_index(V,1)
    n_upper = n + 5
    n_lower = n - 5
    V_integral = np.trapz(V[n_lower:n_upper],t[n_lower:n_upper])
    
    DP = P[n_upper] - P[n_lower]
    
    R = 1.5 * V_integral / DP    
    
    
    P_leakage = scipy.integrate.cumtrapz(V/R,t)
    P_leakage = np.append(P_leakage,P_leakage[-1])
    P_corrected = P - P_leakage
    

    
    
    #major_loop
    recentering_slice = voltage_slice(V,1,3)
    
    
    #new_major = np.vstack((,))
    new_major_slice = voltage_slice(V,2,6)
    #P_return = P_corrected[zero_indices[1]:zero_indices[3]]
    #V_return = V[zero_indices[1]:zero_indices[3]]
    
    #All of the loop
    all_slice = np.index_exp[:]
    
    if no_correction:
        #print('No leakage correction')
        P_return = P
        
    else:
        P_return = P_corrected
    
    #Recentering
    major_loop = P_corrected[recentering_slice]
    
    shift = np.mean((major_loop[0],major_loop[-1]))
    if np.isnan(shift):
        pdb.set_trace()
    P_return = P_return - shift

    return_slice = upper_slice
    
    
    if major_slice:
        return_slice = new_major_slice
    
    P_return = P_return[return_slice]
    V_return = V[return_slice]
    
    
    return (P_return,V_return)


In [12]:
def getCurve(x,y):
    p = np.polyfit(x,y,2)
    curv = 2*p[0]
    return np.mean(curv)

In [13]:
def find_R(P,V,t):
    threshold = np.mean(np.abs(np.diff(V)))*1.5
    DP_array = np.array([])
    VInt_array = np.array([])
    
    for n in np.arange(P.size):
        if n * 3 >= P.size:
            break
         
        candidates = np.where(np.abs(V-V[n])<threshold)[0]
        closest_ind = np.max(candidates)
        
        DP_array = np.append(DP_array,P[closest_ind]-P[n])
        VInt = np.trapz(V[n:closest_ind+1],t[n:closest_ind+1])
        VInt_array = np.append(VInt_array,VInt)
        
        
        
    R = VInt_array / DP_array
    if np.any(np.isnan(R)):
        print(np.sum(np.isnan(R)))
        R = R[np.logical_not(np.isnan(R))]
    
    return R

In [14]:
def second_div(x,y):
    return np.diff(np.diff(y) / np.diff(x)) / np.diff(info_loss(x,1))

In [15]:
def find_R2(P,V,t):
    
    DP = P[-1] - P[0]
    VInt = np.trapz(V,t)
    R = VInt/DP

    return R

In [16]:
def extractPVt(dataset):
    #df_temp = pd.read_table(filename,engine='python')
    data = np.array(dataset)
    
    
    P = np.copy(data[:,1])
    V = np.copy(data[:,0])
    t = np.copy(data[:,2])
    
    sample_area = dataset.attrs['Area / cm^2']
    
    P = P / sample_area #micro C per cm^2
    P = P * 1e-2 #convert to SI
    
    thickness = 300e-6 #in m
    V = V / thickness #To field instead of voltage
    
    return (P,V,t)

def extractT(obj):
    return obj.attrs['Temperature']

#File diagnosis
def diagnose(P,V,t):
    plt.subplot(211)
    plt.plot(t,V)
    plt.ylim([np.min(V), np.max(V)])
    plt.xlim([np.min(t),np.max(t)])

    plt.subplot(212)
    plt.plot(t,P)
    plt.ylim([np.min(P),np.max(P)])
    plt.xlim([np.min(t),np.max(t)])

    plt.show()

In [17]:
#Create T_array (only good files)
T_array = np.array([])
for name,obj in grp.items():
    P,V,t = extractPVt(obj)
    try:
        a = voltage_slice(V,1,5)
    except:
        continue
    T = extractT(obj)
    T_array = np.append(T_array,T)

T_array = np.sort(T_array)

badTs = []
T_array = np.array([T for T in T_array if T not in badTs])
#T_array = T_array[(T_array<450) | (T_array>480)]

In [18]:
T_array

array([ 100.,  200.,  250.,  300.,  360.,  380.,  400.,  410.,  420.,
        430.,  440.,  445.,  450.,  455.,  460.,  465.,  470.,  475.,
        480.,  485.,  490.,  495.,  500.,  505.,  510.,  515.,  520.,
        530.,  535.,  540.,  545.,  550.,  555.,  560.,  565.,  570.])

In [20]:
for name,obj in grp.items():
    T = extractT(obj)
    if T not in T_array:
        continue
    P,V,t = extractPVt(obj)
    plt.plot(t,V)
    zeros = findZeros(V)
    plt.plot(t[zeros],V[zeros],'o')
plt.show()

In [36]:
for name,obj in grp.items():
    T = extractT(obj)
    if T not in T_array:
        continue
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t,no_correction=True)
    p = np.polyfit(V_new,P_new,2)
    plt.plot(V_new,P_new,'b')
    plt.plot(V_new,np.polyval(p,V_new))
    plt.title(T)
    plt.show()

In [49]:
for name,obj in grp.items():
    T = extractT(obj)
    if T not in T_array:
        continue
    P,V,t = extractPVt(obj)
    plt.plot(V,P)
    plt.show()


In [72]:
#Make sure you use upper_slice
for name,obj in grp.items():
    T = extractT(obj)
    if T not in T_array:
        continue
    T_color = (T-np.min(T_array))/(np.max(T_array)-np.min(T_array))
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t,no_correction=False)
    plt.plot(V_new,P_new,color=(T_color*1,T_color*0,T_color*0))

#plt.ylim([0,100])
#plt.xlim([0,500])
plt.title(grp.name)
plt.xlabel('E / V m$^{-1}$')
plt.ylabel('P / C m$^{-2}$')
#plt.savefig(save_dir+directory_name+'PVupper.png',dpi=300)
plt.show()

In [73]:
#Plot 5 loops across the temperature range
#Make sure you use upper_slice
loops = T_array[::T_array.size//5]
ind = 10
loops = T_array
for name,obj in grp.items():

    T = extractT(obj)
    if T not in loops:
        continue
    T_color = (T-np.min(T_array))/(np.max(T_array)-np.min(T_array))
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t,no_correction=False,major_slice=True)
    P_nc,V_nc = leakage_correction(P,V,t,no_correction=True,major_slice=True)
    P_upper,V_upper = leakage_correction(P,V,t)
    
    #df_temp = pd.DataFrame(data=np.transpose(np.vstack((P_new,V_new))), columns=['P / SI', 'V / SI'])
    #df_temp.to_csv(save_dir+grp_name+'loop'+str(T)+'.csv',index_label=False,index=False)
    plt.plot(V_new,P_new,color=(T_color,T_color*0.3,T_color*0.8))
    plt.plot(V_nc,P_nc)
    #plt.plot(V_upper,P_upper,'r')
    plt.show()


#plt.ylim([0,100])
#plt.xlim([0,500])
plt.title(grp.name)
plt.xlabel('E / V m$^{-1}$')
plt.ylabel('P / C m$^{-2}$')
#plt.savefig(save_dir+directory_name+'PVupper.png',dpi=300)
plt.show()

In [149]:
loops = T_array[::T_array.size//6]
for name,obj in grp.items():
    T = extractT(obj)
    if T not in loops:
        continue
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t,major_slice=True)
    df = pd.DataFrame(data={'P / C m^-2': P_new, 'E / V m^-1':V_new})
    df.to_csv(save_dir+grp_name+'loop'+str(T)+'.csv',index=False,index_label=False)
        

In [65]:
#Find absolute least extrema

V_max_array = np.array([]) #The maximum values of V in single files 
V_min_array = np.array([]) #The minimum ---------------------------
for name,obj in grp.items():
    if extractT(obj) not in T_array:
        continue
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t)
    
    V_max_array = np.append(V_max_array,np.max(V_new))
    V_min_array = np.append(V_min_array,np.min(V_new))
#assert T_array.size==V_max_array.size

In [66]:
V_lower = np.max(V_min_array)
V_upper = np.min(V_max_array)


no_of_points = 200
no_of_T_points = 500
V_range = np.linspace(V_lower,V_upper,no_of_points)
T_vec,V_vec = np.meshgrid(T_array,V_range)

P_mesh = np.empty(T_vec.shape)

for name,obj in grp.items():
    if extractT(obj) not in T_array:
        continue
    P,V,t = extractPVt(obj)
    P_new,V_new = leakage_correction(P,V,t)    
    
    
    T = extractT(obj)
    n = np.where(T_array==T)[0][0]
    assert np.max(V_range) <= np.max(V_new)
    assert np.min(V_range) >= np.min(V_new)

    sorted_indices = np.argsort(V_new)
    V_new = V_new[sorted_indices]
    P_new = P_new[sorted_indices]
    P_mesh[:,n] = np.interp(V_range,V_new,P_new)

In [67]:
P_mesh_smooth = np.copy(P_mesh)
T_vec_smooth = np.copy(T_vec)
V_vec_smooth = np.copy(V_vec)

In [38]:
#Pad P_mesh, T_vec and V_vec before smoothing
pw = ((0,0),(5,5)) #Pad width
P_mesh = np.pad(P_mesh,pw,'edge')
T_vec  = np.pad(T_vec,pw,'linear_ramp', end_values=[np.min(T_vec)*0.9,np.max(T_vec)*1.1])
V_vec = np.pad(V_vec,pw,'edge')

In [46]:
no_of_T_points = 200
smoothing = 5e-2
T_new = np.linspace(T_vec[0,0],T_vec[0,-1],no_of_T_points)
P_mesh_smooth = np.empty((P_mesh.shape[0],no_of_T_points))
T_vec_smooth,V_vec_smooth = np.meshgrid(T_new,V_range)


weights = np.arange(T_vec[0].size)
weights = (weights-T_vec[0].size//2)**2/100 + 10
#plt.plot(weights)
#plt.show()

for k in np.arange(P_mesh.shape[0]):
    P = P_mesh[k]
    T = T_vec[k]    
    tck = scipy.interpolate.splrep(T,P,s=smoothing)
    P_new = scipy.interpolate.splev(T_new,tck)
    if np.any(np.isnan(P_new)):
        pdb.set_trace()
    P_mesh_smooth[k] = P_new




In [68]:
#P-T plots at different fields
def plotPT():
    no_of_lines_to_show = 10
    for n in np.arange(P_mesh.shape[0]):
        if n % (P_mesh.shape[0] // no_of_lines_to_show) != 0:
            continue
        my_color = (np.random.rand(),np.random.rand(),np.random.rand())
        plt.plot(T_vec[n],100*P_mesh[n], 'x',color=my_color)
        plt.plot(T_vec_smooth[n],100*P_mesh_smooth[n],'-',color=my_color)

    plt.title(grp.name)
    plt.xlabel('T / K')
    plt.ylabel('P / $\mu$ C cm$^{-2}$')

    plt.xlim([T_array[0],T_array[-1]])
    #plt.ylim([0,250])
    #plt.savefig(save_dir+directory_name+'PT.png')
plotPT()
plt.show()

#plt.ylim([0.6,1.2])

In [48]:
#Taking the derivative with respect to T (Confirmed)
dPdT = np.diff(P_mesh_smooth,axis=1) / np.diff(T_vec_smooth,axis=1)
T_vec_d = scipy.signal.convolve2d(T_vec_smooth,[np.ones(2)/2],mode='valid')
V_vec_d =  scipy.signal.convolve2d(V_vec_smooth,[np.ones(2)/2],mode='valid')

In [49]:
#Integrating dPdT wrt. E' (dummy) from  E_lower to E (NOT Confirmed)
assert np.all(np.diff(V_vec_d[:,0]) > 0) #Make sure E is always increasing
P_integrated = scipy.integrate.cumtrapz(dPdT,V_vec_d,axis=0)
V_vec_int = scipy.signal.convolve2d(V_vec_d,np.ones((2,1))/2,mode='valid')
T_vec_int = scipy.signal.convolve2d(T_vec_d,np.ones((2,1))/2,mode='valid')
rho = 8100
DS = P_integrated / rho

In [50]:
no_of_lines_to_show = 10
plt.subplot(211)
plotPT()
plt.xlim([T_array[0],T_array[-1]])
plt.subplot(212)
for n in np.arange(P_integrated.shape[0]):
    if n % (P_integrated.shape[0] // no_of_lines_to_show) != 0:
        continue
    V_color = V_vec_int[n,0]
    plt.plot(T_vec_int[n],DS[n],'-')

plt.xlabel('T/K')
plt.ylabel('$\Delta S$ / J K$^{-1}$ kg$^{-1}$')
#plt.ylim([-8,10])
plt.xlim([T_array[0],T_array[-1]])

plt.show()

In [55]:
P_mesh.shape

(200, 32)

In [56]:
T_array.size

32

In [54]:
P_mesh = P_mesh[:,5:-5]
T_vec = T_vec[:,5:-5]
V_vec = V_vec[:,5:-5]

In [58]:
save_dir = r'/home/william/Dropbox/Materials Internship/Presentations/PZT indirect/PLZT new doping/'

In [57]:
#Saving P_mesh etc
df_P = pd.DataFrame(data=P_mesh)
df_P.to_csv(save_dir+grp_name+'P.csv')

df_T = pd.DataFrame(data=T_vec)
df_T.to_csv(save_dir+grp_name+'T.csv')

df_V = pd.DataFrame(data=V_vec)
df_V.to_csv(save_dir+grp_name+'E.csv')



df_P_smooth = pd.DataFrame(data=P_mesh_smooth)
df_P_smooth.to_csv(save_dir+grp_name+'P_smooth.csv')

df_T_smooth = pd.DataFrame(data=T_vec_smooth)
df_T_smooth.to_csv(save_dir+grp_name+'T_smooth.csv')

df_V_smooth = pd.DataFrame(data=V_vec_smooth)
df_V_smooth.to_csv(save_dir+grp_name+'E_smooth.csv')



df_P_integrated = pd.DataFrame(data=P_integrated)
df_P_integrated.to_csv(save_dir+grp_name+'DS.csv')

df_T_int = pd.DataFrame(data=T_vec_int)
df_T_int.to_csv(save_dir+grp_name+'T_int.csv')

df_V_int = pd.DataFrame(data=V_vec_int)
df_V_int.to_csv(save_dir+grp_name+'E_int.csv')



In [54]:
P_mesh.shape

(200, 37)

In [54]:
no_of_T_points = 200

T_new = np.linspace(T_vec[0,0],T_vec[0,-1],no_of_T_points)
P_mesh_smooth = np.empty((P_mesh.shape[0],no_of_T_points))

for k in np.arange(P_mesh.shape[0]):
    P = P_mesh[k]
    T = T_vec[k]
    tck = scipy.interpolate.splrep(T,P,s=0.0029)
    P_new = scipy.interpolate.splev(T_new,tck)
    if np.any(np.isnan(P_new)):
        pdb.set_trace()
    P_mesh_smooth[k] = P_new



In [69]:
#Ensure that there are PT lines crossing
step1 = np.diff(P_mesh_smooth,axis=0)
step2 = np.all(step1>0,axis=1)
good_indices = np.where(step2)[0]+1


P_mesh_smooth = P_mesh_smooth[good_indices]
T_vec_smooth = T_vec_smooth[good_indices]
V_vec_smooth = V_vec_smooth[good_indices]

In [70]:
for n in np.arange(P_mesh_smooth.shape[0]):
    plt.plot(T_new,P_mesh_smooth[n])
plt.show()

In [50]:
f

<HDF5 file "indirect_data.h5" (mode r)>

In [51]:
f.close()

In [37]:
x = np.linspace(1,10,100)

In [39]:
y = 5*x**2 - 3*x + 10

array([ 10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,
        10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.,  10.])